In [8]:
#!/usr/bin/env python3
import csv
import datetime
import requests

FILE_URL="http://marga.com.ar/employees-with-date.csv"

def get_start_date():
    """Interactively get the start date to query for."""

    print()
    print('Getting the first start date to query for.')
    print()
    print('The date must be greater than Jan 1st, 2018')
    year = int(input('Enter a value for the year: '))
    month = int(input('Enter a value for the month: '))
    day = int(input('Enter a value for the day: '))
    print()

    return datetime.datetime(year, month, day)

def get_file_content(url):
    """Download the file over the internet and
    convert needed columns into sorted dictionary."""

    my_dict = {}
    with requests.get(url, stream=True) as r:
        lines = (line.decode('utf-8') for line in r.iter_lines())
        reader = csv.reader(lines)
        next(reader)
        for row in reader:
            my_dict[row[3]] = [row[0] + " " + row[1]]
    return dict(sorted(my_dict.items()))

def get_same_or_newer(start_date, data):
    """Go through all the data and find the employees that started on the given date, or the closest one. Do not include dates after today."""

    my_date_employees = {}
    for key in data:
        name = data[key]
        date = datetime.datetime.strptime(key, '%Y-%m-%d')
        if date >= start_date and date <= datetime.datetime.today():
            my_date_employees[date] = name
    return my_date_employees

def list_newer(start_date, data):
    employees = get_same_or_newer(start_date, data)
    for key in employees:
        print("Started on {}: {}".format(key.strftime("%b %d, %Y"), employees[key]))

def main():
    data = get_file_content(FILE_URL)
    start_date = get_start_date()
    list_newer(start_date, data)

if __name__ == "__main__":
    main()

ConnectionError: HTTPConnectionPool(host='marga.com.ar', port=80): Max retries exceeded with url: /employees-with-date.csv (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb9ac797290>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [3]:
#!/usr/bin/env python3
import csv
import datetime
import requests

FILE_URL="http://marga.com.ar/employees-with-date.csv"

def get_start_date():
    """Interactively get the start date to query for."""

    print()
    print('Getting the first start date to query for.')
    print()
    print('The date must be greater than Jan 1st, 2018')
    year = int(input('Enter a value for the year: '))
    month = int(input('Enter a value for the month: '))
    day = int(input('Enter a value for the day: '))
    print()
    print("hello",datetime.datetime(year, month, day))
    return datetime.datetime(year, month, day)

def get_file_lines(url):
    """Returns the lines contained in the file at the given URL"""

    # Download the file over the internet
    response = requests.get(url, stream=True)

    # Decode all lines into strings
    lines = []
    for line in response.iter_lines():
        lines.append(line.decode("UTF-8"))
    return lines

def get_same_or_newer(start_date):
    """Returns the employees that started on the given date, or the closest one."""
    data = get_file_lines(FILE_URL)
    reader = csv.reader(data[1:])

    # We want all employees that started at the same date or the closest newer
    # date. To calculate that, we go through all the data and find the
    # employees that started on the smallest date that's equal or bigger than
    # the given start date.
    min_date = datetime.datetime.today()
    min_date_employees = []
    for row in reader: 
        row_date = datetime.datetime.strptime(row[3], '%Y-%m-%d')

        # If this date is smaller than the one we're looking for,
        # we skip this row
        if row_date < start_date:
            continue

        # If this date is smaller than the current minimum,
        # we pick it as the new minimum, resetting the list of
        # employees at the minimal date.
        if row_date < min_date:
            min_date = row_date
            min_date_employees = []

        # If this date is the same as the current minimum,
        # we add the employee in this row to the list of
        # employees at the minimal date.
        if row_date == min_date:
            min_date_employees.append("{} {}".format(row[0], row[1]))

    return min_date, min_date_employees

def list_newer(start_date):
    while start_date < datetime.datetime.today():
        start_date, employees = get_same_or_newer(start_date)
        print("Started on {}: {}".format(start_date.strftime("%b %d, %Y"), employees))

        # Now move the date to the next one
        start_date = start_date + datetime.timedelta(days=1)

def main():
    start_date = get_start_date()
    list_newer(start_date)

if __name__ == "__main__":
    main()



Getting the first start date to query for.

The date must be greater than Jan 1st, 2018
Enter a value for the year: 2020
Enter a value for the month: 04
Enter a value for the day: 28

hello 2020-04-28 00:00:00


In [7]:
#!/usr/bin/env python3
import csv
import datetime
import requests

FILE_URL="http://marga.com.ar/employees-with-date.csv"

def get_start_date():
    """Interactively get the start date to query for."""

    print()
    print('Getting the first start date to query for.')
    print()
    print('The date must be greater than Jan 1st, 2018')
    year = int(input('Enter a value for the year: '))
    month = int(input('Enter a value for the month: '))
    day = int(input('Enter a value for the day: '))
    print()
    print("hello",datetime.datetime(year, month, day))
    return datetime.datetime(year, month, day)

def get_file_lines(url):
    """Returns the lines contained in the file at the given URL"""

    # Download the file over the internet
    response = requests.get(url, stream=True)

    # Decode all lines into strings
    lines = []
    for line in response.iter_lines():
        lines.append(line.decode("UTF-8"))
    return lines

def get_same_or_newer(start_date, data):
    """Returns the employees that started on the given date, or the closest one."""
    
    reader = csv.reader(data[1:])

    # We want all employees that started at the same date or the closest newer
    # date. To calculate that, we go through all the data and find the
    # employees that started on the smallest date that's equal or bigger than
    # the given start date.
    min_date = datetime.datetime.today()
    min_date_employees = []
    for row in reader: 
        row_date = datetime.datetime.strptime(row[3], '%Y-%m-%d')

        # If this date is smaller than the one we're looking for,
        # we skip this row
        if row_date < start_date:
            continue

        # If this date is smaller than the current minimum,
        # we pick it as the new minimum, resetting the list of
        # employees at the minimal date.
        if row_date < min_date:
            min_date = row_date
            min_date_employees = []

        # If this date is the same as the current minimum,
        # we add the employee in this row to the list of
        # employees at the minimal date.
        if row_date == min_date:
            min_date_employees.append("{} {}".format(row[0], row[1]))

    return min_date, min_date_employees

def list_newer(start_date,data):
    while start_date < datetime.datetime.today():
        start_date, employees = get_same_or_newer(start_date,data)
        print("Started on {}: {}".format(start_date.strftime("%b %d, %Y"), employees))

        # Now move the date to the next one
        #start_date = start_date + datetime.timedelta(days=1)

def main():
    data = get_file_lines(FILE_URL)
    start_date = get_start_date()
    list_newer(start_date, data)

if __name__ == "__main__":
    main()


2020-04-28 00:00:00
2020-04-29 00:00:00


In [ ]:
import time
import csv
import datetime
import requests

page = ''
while page == '':
    try:
        page = requests.get("http://marga.com.ar/employees-with-date.csv", stream=True)
        print(page)
        break
    except:
        print("Connection refused by the server..")
        print("Let me sleep for 5 seconds")
        print("ZZzzzz...")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue

Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
